# Milestone 3: Interactive Visualization and Analysis

In order to display the forecasts of our CNN-based stock market model, this milestone focuses on developing an interactive dashboard. The dashboard gives financial experts the ability to investigate stock price patterns and dynamically compare real and projected values. Among the essential features are:

- Data can be filtered using date ranges and stock symbols.
- Visualizing actual vs. predicted prices.
- Displaying metrics for performance such as Mean Squared Error (MSE) and Mean Absolute Error (MAE).

Plotly and Dash will be used to create this interactive visualization.


## Importing Libraries and Preparing Data

Load the required libraries and get the dataset ready for visualization in this section. 
The data that we load or simulate includes the following columns:
- **Date**: The time period for the stock prices.
- **Stock_Symbol**: The stock being analyzed (e.g., AAPL, GOOGL).
- **Actual_Price**: The observed stock price.
- **Predicted_Price**: The stock price predicted by our CNN model.

Data is organized so that it can be filtered by date and stock symbol.


In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from dash import Dash, dcc, html, Input, Output

# Load predicted data (replace with your actual data)
df_predictions = pd.DataFrame({
    'Date': pd.date_range(start='2023-01-01', periods=100),
    'Stock_Symbol': ['AAPL'] * 50 + ['GOOGL'] * 50,
    'Actual_Price': np.random.uniform(100, 500, 100),
    'Predicted_Price': np.random.uniform(100, 500, 100)
})


## Building the Dashboard Layout

The components of the dashboard are as follows:
1. **Dropdown Menu**: Enables users to choose a particular stock symbol for analysis.
2. **Date Range Slider**: Allows data to be filtered within a given time frame.
3. **Interactive Graph**: Displays the comparison between actual and predicted stock prices.
4. **Summary Metrics**: Evaluates the accuracy of the model by providing key performance indicators such as MAE and MSE.

These interactive features guarantee that the dashboard is easy to use and offers useful information.


In [2]:
# Initialize the Dash app
app = Dash(__name__)

# Define the layout
app.layout = html.Div([
    html.H1("Stock Market Prediction Dashboard", style={'textAlign': 'center'}),
    html.Div([
        html.Label("Select Stock Symbol:"),
        dcc.Dropdown(
            id='stock-dropdown',
            options=[{'label': symbol, 'value': symbol} for symbol in df_predictions['Stock_Symbol'].unique()],
            value='AAPL',  # Default value
            style={'width': '50%'}
        ),
        html.Label("Select Date Range:"),
        dcc.RangeSlider(
            id='date-slider',
            min=0,
            max=len(df_predictions['Date']) - 1,
            value=[0, len(df_predictions['Date']) - 1],
            marks={i: date.strftime('%Y-%m-%d') for i, date in enumerate(df_predictions['Date'])},
            step=1
        )
    ], style={'margin': '20px'}),
    dcc.Graph(id='price-graph'),
    html.Div(id='summary-metrics', style={'margin': '20px'})
])

## Adding Interactivity to the Dashboard

Update dynamically by utilizing Dash's callback functions:
- The primary chart is based on date ranges and stock symbols chosen by the user.
- To represent the chosen data subset, the summary metrics (MAE and MSE) are used.

This enables financial analysts to efficiently examine particular stocks and time periods.


In [3]:
@app.callback(
    [Output('price-graph', 'figure'),
     Output('summary-metrics', 'children')],
    [Input('stock-dropdown', 'value'),
     Input('date-slider', 'value')]
)
def update_dashboard(stock_symbol, date_range):
    # Filter data based on stock symbol and date range
    filtered_data = df_predictions[
        (df_predictions['Stock_Symbol'] == stock_symbol) &
        (df_predictions.index >= date_range[0]) &
        (df_predictions.index <= date_range[1])
    ]
    
    # Create the graph
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=filtered_data['Date'], y=filtered_data['Actual_Price'],
        mode='lines+markers', name='Actual Price'
    ))
    fig.add_trace(go.Scatter(
        x=filtered_data['Date'], y=filtered_data['Predicted_Price'],
        mode='lines+markers', name='Predicted Price'
    ))
    fig.update_layout(title="Stock Prices: Actual vs Predicted",
                      xaxis_title='Date', yaxis_title='Stock Price',
                      legend_title="Legend")
    
    # Calculate summary metrics
    mae = np.mean(np.abs(filtered_data['Actual_Price'] - filtered_data['Predicted_Price']))
    mse = np.mean((filtered_data['Actual_Price'] - filtered_data['Predicted_Price'])**2)
    metrics = [
        html.P(f"Mean Absolute Error (MAE): {mae:.2f}"),
        html.P(f"Mean Squared Error (MSE): {mse:.2f}")
    ]
    
    return fig, metrics


## Running the Dashboard

To render the interactive dashboard, lastly launch the Dash app.



In [4]:
if __name__ == '__main__':
    app.run_server(debug=True, port=8051)


# Conclusion

Financial analysts can use the interactive visualization dashboard included in this milestone to:
- Explore CNN's predictions in an interactive manner.
- Sort data according to date range and stock symbol.
- Visually compare projected and actual stock prices.
- Analyze the model's performance with important metrics.

This represents a major advancement in the development of a useful and approachable financial analysis tool.
